In [2]:
import glob
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import keras as k

Using TensorFlow backend.


In [3]:
data = pd.read_csv("stroke.csv", index_col=0)

In [4]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [5]:
data = data.drop("ever_married" , axis=1)

In [6]:
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,Male,67.0,0,1,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Self-employed,Rural,174.12,24.0,never smoked,1


In [7]:
column_req = ["gender", "age", "hypertension", "work_type", "Residence_type", "avg_glucose_level", "bmi", "smoking_status","stroke"]

In [8]:
for column in data.columns:
    if data[column].dtype == np.number:
        continue
    data[column] = LabelEncoder().fit_transform( data[column] )

In [9]:
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.6,1,1
51676,0,61.0,0,0,3,0,202.21,NaN,2,1
31112,1,80.0,0,1,2,0,105.92,32.5,2,1
60182,0,49.0,0,0,2,1,171.23,34.4,3,1
1665,0,79.0,1,0,3,0,174.12,24.0,2,1


In [10]:
data = data.fillna(data.mean())
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.600000,1,1
51676,0,61.0,0,0,3,0,202.21,28.893237,2,1
31112,1,80.0,0,1,2,0,105.92,32.500000,2,1
60182,0,49.0,0,0,2,1,171.23,34.400000,3,1
1665,0,79.0,1,0,3,0,174.12,24.000000,2,1


In [11]:
X = data.drop(["stroke"], axis=1)
Y = data["stroke"]

In [12]:
X.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
id,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.600000,1
51676,0,61.0,0,0,3,0,202.21,28.893237,2
31112,1,80.0,0,1,2,0,105.92,32.500000,2
60182,0,49.0,0,0,2,1,171.23,34.400000,3
1665,0,79.0,1,0,3,0,174.12,24.000000,2


In [13]:
Y.head()


id
9046     1
51676    1
31112    1
60182    1
1665     1
Name: stroke, dtype: int64

In [14]:
x_scaler = MinMaxScaler()
x_scaler.fit(X)
column_names = X.columns
X[column_names]=x_scaler.transform(X)

In [15]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size= 0.2, shuffle = True)

In [16]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0))) 

Before OverSampling, counts of label '1': 194
Before OverSampling, counts of label '0': 3894 



In [17]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_train_res, Y_train_res = sm.fit_sample(X_train, Y_train.ravel()) 

In [18]:
print("After OverSampling, counts of label '1': {}".format(sum(Y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(Y_train_res == 0))) 

After OverSampling, counts of label '1': 3894
After OverSampling, counts of label '0': 3894


# Build model

In [19]:
model = Sequential()

In [20]:
model.add(Dense(256,input_dim=len(X.columns),kernel_initializer = k.initializers.random_normal(seed = 13), activation = 'relu'))
model.add(Dense(1,activation = 'hard_sigmoid'))

# Compiling the model

In [21]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model

In [26]:
history = model.fit(X_train_res, Y_train_res, epochs = 2000, batch_size = X_train.shape[0])

Epoch 1/2000
7788/7788 [==============================] - 0s 2us/step - loss: 0.9826 - accuracy: 0.5017
Epoch 2/2000
7788/7788 [==============================] - 0s 3us/step - loss: 0.6837 - accuracy: 0.5401
Epoch 3/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.5531 - accuracy: 0.6659
Epoch 4/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.5295 - accuracy: 0.7657
Epoch 5/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.6277 - accuracy: 0.7664
Epoch 6/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.6960 - accuracy: 0.7503
Epoch 7/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.6057 - accuracy: 0.7627
Epoch 8/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.5339 - accuracy: 0.7812
Epoch 9/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.5134 - accuracy: 0.7906
Epoch 10/2000
7788/7788 [==============================] - 0s 4u

7788/7788 [==============================] - 0s 4us/step - loss: 0.4367 - accuracy: 0.8020
Epoch 79/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4364 - accuracy: 0.8019
Epoch 80/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4361 - accuracy: 0.8017
Epoch 81/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4359 - accuracy: 0.8019
Epoch 82/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4356 - accuracy: 0.8017
Epoch 83/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4354 - accuracy: 0.8017
Epoch 84/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4352 - accuracy: 0.8016
Epoch 85/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4349 - accuracy: 0.8015
Epoch 86/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4347 - accuracy: 0.8014
Epoch 87/2000
7788/7788 [==============================] - 0s 4us/ste

7788/7788 [==============================] - 0s 4us/step - loss: 0.4249 - accuracy: 0.8055
Epoch 156/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4248 - accuracy: 0.8046
Epoch 157/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4247 - accuracy: 0.8043
Epoch 158/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4246 - accuracy: 0.8055
Epoch 159/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4245 - accuracy: 0.8053
Epoch 160/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4244 - accuracy: 0.8051
Epoch 161/2000
7788/7788 [==============================] - 0s 3us/step - loss: 0.4243 - accuracy: 0.8052
Epoch 162/2000
7788/7788 [==============================] - 0s 3us/step - loss: 0.4242 - accuracy: 0.8053
Epoch 163/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4241 - accuracy: 0.8056
Epoch 164/2000
7788/7788 [==============================] - 0

7788/7788 [==============================] - 0s 4us/step - loss: 0.4178 - accuracy: 0.8080
Epoch 233/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4177 - accuracy: 0.8087
Epoch 234/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4176 - accuracy: 0.8084
Epoch 235/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4176 - accuracy: 0.8086
Epoch 236/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4175 - accuracy: 0.8086
Epoch 237/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4174 - accuracy: 0.8088
Epoch 238/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4173 - accuracy: 0.8088
Epoch 239/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.4172 - accuracy: 0.8087
Epoch 240/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4171 - accuracy: 0.8087
Epoch 241/2000
7788/7788 [==============================] - 0

7788/7788 [==============================] - 0s 4us/step - loss: 0.4114 - accuracy: 0.8118
Epoch 310/2000
7788/7788 [==============================] - 0s 3us/step - loss: 0.4113 - accuracy: 0.8115
Epoch 311/2000
7788/7788 [==============================] - 0s 3us/step - loss: 0.4112 - accuracy: 0.8111
Epoch 312/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4111 - accuracy: 0.8111
Epoch 313/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4110 - accuracy: 0.8110
Epoch 314/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4109 - accuracy: 0.8114
Epoch 315/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4109 - accuracy: 0.8111
Epoch 316/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4108 - accuracy: 0.8118
Epoch 317/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.4107 - accuracy: 0.8116
Epoch 318/2000
7788/7788 [==============================] - 0

...

7788/7788 [==============================] - 0s 4us/step - loss: 0.3392 - accuracy: 0.8455
Epoch 1686/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3392 - accuracy: 0.8460
Epoch 1687/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3392 - accuracy: 0.8458
Epoch 1688/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3391 - accuracy: 0.8458
Epoch 1689/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3390 - accuracy: 0.8458
Epoch 1690/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3390 - accuracy: 0.8460
Epoch 1691/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3390 - accuracy: 0.8459
Epoch 1692/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3389 - accuracy: 0.8463
Epoch 1693/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3388 - accuracy: 0.8460
Epoch 1694/2000
7788/7788 [==========================

7788/7788 [==============================] - 0s 4us/step - loss: 0.3354 - accuracy: 0.8482
Epoch 1761/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3354 - accuracy: 0.8480
Epoch 1762/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3352 - accuracy: 0.8487
Epoch 1763/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3352 - accuracy: 0.8494
Epoch 1764/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3349 - accuracy: 0.8491
Epoch 1765/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3349 - accuracy: 0.8482
Epoch 1766/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3349 - accuracy: 0.8491
Epoch 1767/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3347 - accuracy: 0.8487
Epoch 1768/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3348 - accuracy: 0.8481
Epoch 1769/2000
7788/7788 [==========================

7788/7788 [==============================] - 0s 5us/step - loss: 0.3309 - accuracy: 0.8512
Epoch 1837/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3309 - accuracy: 0.8517
Epoch 1838/2000
7788/7788 [==============================] - 0s 6us/step - loss: 0.3308 - accuracy: 0.8517
Epoch 1839/2000
7788/7788 [==============================] - 0s 6us/step - loss: 0.3308 - accuracy: 0.8517
Epoch 1840/2000
7788/7788 [==============================] - 0s 6us/step - loss: 0.3308 - accuracy: 0.8526
Epoch 1841/2000
7788/7788 [==============================] - 0s 6us/step - loss: 0.3306 - accuracy: 0.8521
Epoch 1842/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3311 - accuracy: 0.8511
Epoch 1843/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3313 - accuracy: 0.8509
Epoch 1844/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3316 - accuracy: 0.8504
Epoch 1845/2000
7788/7788 [==========================

7788/7788 [==============================] - 0s 5us/step - loss: 0.3271 - accuracy: 0.8531
Epoch 1913/2000
7788/7788 [==============================] - 0s 6us/step - loss: 0.3270 - accuracy: 0.8541
Epoch 1914/2000
7788/7788 [==============================] - 0s 6us/step - loss: 0.3271 - accuracy: 0.8554
Epoch 1915/2000
7788/7788 [==============================] - 0s 6us/step - loss: 0.3269 - accuracy: 0.8536
Epoch 1916/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3270 - accuracy: 0.8531
Epoch 1917/2000
7788/7788 [==============================] - 0s 4us/step - loss: 0.3268 - accuracy: 0.8549
Epoch 1918/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3269 - accuracy: 0.8548
Epoch 1919/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3269 - accuracy: 0.8541
Epoch 1920/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3266 - accuracy: 0.8541
Epoch 1921/2000
7788/7788 [==========================

7788/7788 [==============================] - 0s 6us/step - loss: 0.3217 - accuracy: 0.8555
Epoch 1989/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3219 - accuracy: 0.8546
Epoch 1990/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3217 - accuracy: 0.8564
Epoch 1991/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3215 - accuracy: 0.8564
Epoch 1992/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3216 - accuracy: 0.8550
Epoch 1993/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3215 - accuracy: 0.8552
Epoch 1994/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3215 - accuracy: 0.8572
Epoch 1995/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3213 - accuracy: 0.8553
Epoch 1996/2000
7788/7788 [==============================] - 0s 5us/step - loss: 0.3214 - accuracy: 0.8549
Epoch 1997/2000
7788/7788 [==========================

In [27]:
model.save('ckd.model')

In [29]:
print("shape of train data", X_train_res.shape)
print("shape of test data", X_test.shape)

shape of train data (7788, 9)
shape of test data (1022, 9)
